In [1]:
import plotly          
import plotly.graph_objects as go

import dash
import pandas as pd
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

In [2]:
df = pd.read_csv('cc_data.csv')
#d = ['2021-12-08','2021-12-07','2021-12-06']
#dft = df[df['Date'].isin(d)]
l = [0,1,3,4,5]
dfM = df.iloc[:,l]
dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
#dfM[dfM['Date']=='2021-12-24']
#df.groupby(by='Date').mean()['Diff'][-1:]
dfM
#df.groupby(by='Date').mean()[-1:]['Calculated_Hash']
#df.groupby(by='Date').mean().filter(items=['2021-12-01'], axis=0)['Calculated_Hash']

,Date,Date_Time,Diff,hash,rate
0,0,0,0.000000,Reported_Hash,0
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
...,...,...,...,...,...
3097,2022-01-16,19:50:33.806324,153.572454,Calculated_Hash,19
3098,2022-01-16,20:05:44.231653,0.000000,Calculated_Hash,24
3099,2022-01-16,20:20:54.770765,167.929605,Calculated_Hash,19
3100,2022-01-16,20:36:05.253593,0.000000,Calculated_Hash,48


In [3]:
def inter_func():
  df = pd.read_csv('cc_data.csv')
  l = [0,1,3,4,5]
  df_icb = df.iloc[:,l]
  df_icb = pd.melt(df_icb, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
  return df_icb

df_itv = inter_func()
df_itv

,Date,Date_Time,Diff,hash,rate
0,0,0,0.000000,Reported_Hash,0
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
...,...,...,...,...,...
3097,2022-01-16,19:50:33.806324,153.572454,Calculated_Hash,19
3098,2022-01-16,20:05:44.231653,0.000000,Calculated_Hash,24
3099,2022-01-16,20:20:54.770765,167.929605,Calculated_Hash,19
3100,2022-01-16,20:36:05.253593,0.000000,Calculated_Hash,48


In [4]:
app = dash.Dash(__name__)

#------------------------------------------------------------------------
app.layout = html.Div([
    dcc.Interval(
                id='my_interval',
                disabled=False,     #if True, the counter will no longer update
                interval=1*3000,    #increment the counter n_intervals every interval milliseconds
                n_intervals=0,      #number of times the interval has passed
                max_intervals=20,    #number of times the interval will be fired.
                                    #if -1, then the interval has no limit (the default)
                                    #and if 0 then the interval stops running.
    ),
    dcc.Graph(id='graph-with-drop'),
    html.Div(id='output_data', style={'font-size':36}),
    dcc.Input(id="input_text",type='text'),
    dcc.Graph(id="mybarchart"),

])

#------------------------------------------------------------------------
@app.callback(
    [Output('output_data', 'children'),
     Output('mybarchart', 'figure'),
     Output('graph-with-drop','figure')],
    [Input('my_interval', 'n_intervals')]
)
def update_graph(num):
    """update every 3 seconds"""
    if num==0:
        raise PreventUpdate
    else:
        inter_func()
        y_data=num
        fig=go.Figure(data=[go.Bar(x=[1, 2, 3, 4, 5, 6, 7, 8, 9], y=[y_data]*9)],
                      layout=go.Layout(yaxis=dict(tickfont=dict(size=22)))
        )
        df_intv = inter_func()
        filtered_df = df_intv[df_intv.Date == '2022-01-15']
        fig1 = px.bar(filtered_df, x="Date_Time", y='Diff',
          color="hash", barmode='group')
        fig1.update_layout(transition_duration=500)

    return (y_data,fig,fig1)

#------------------------------------------------------------------------
@app.callback(
    Output('my_interval', 'max_intervals'),
    [Input('input_text', 'value')]
)
def stop_interval(retrieved_text):
    if retrieved_text == 'stop':
        max_intervals = 0
    else:
        raise PreventUpdate

    return (max_intervals)
#------------------------------------------------------------------------

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jan/2022 22:08:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [16/Jan/2022 22:09:00] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [16/Jan/2022 22:09:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2022 22:09:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16